## Ordnance Survey Data Hub Tutorials - Example Applications

# Using the OS Places API with ipyleaflet

This is a simple Python/Jupyter Notebook application, demonstrating how the OS Places API can be used to populate selected 
point features within an interactive ipyleaflet (https://ipyleaflet.readthedocs.io/en/latest/) map.  Aside from using the OS Places API itself, the application also makes use of the OS Maps API, as a source of contextual background mapping. 

This example queries addresses that fall within a 100m radius of a specified point in the centre of Abingdon-on-Thames.  The OS Places API is able to return two distinct datasets, 'DPA' data, derived from Royal Mail's delivery point database, and 'LPI' addresses, which are sourced from local authorities.  Either one of these can be specified in the request URL ('DPA' is the default), or both can be returned together, which is what we'll demonstrate in this example.  Thus, we'll be treating each dataset type independently and representing each within their own layer.

ipyleaflet is able to support GeoJSON, so one option for approaching this would be to convert the returned JSON data into GeoJSON and use it directly.  Unfortunately, we've not found a way of styling the GeoJSON marker symbols...it just uses the default Leaflet blue map markers.  As this demonstration requires us two use two separate layers, that doesn't really work if the markers all look the same.  As such, we've chosen to ignore the GeoJSON layer type for this demonstation and use the configurable AwesomeIcon layers instead.

The OS Places API, one of the many services available from the OS Data Hub, is an address look-up service based upon Ordnance Survey's AddressBase Premium product.  It's available on the Premium Plan and Public Sector Plan.

In [1]:
from datetime import datetime
from ipyleaflet import Map, WidgetControl, AwesomeIcon, Marker, MarkerCluster
from urllib.parse import unquote, urlencode
from ipywidgets import Image
import requests
import IPython

Use of the OS APIs requires a service key. Sign up for yours at https://osdatahub.os.uk/ and enter it below.

In [2]:
# OS Data Hub project API key (key from guy.heathcote@os.uk)
key = 'YOUR API KEY HERE'

The following cells set up a basic ipyleaflet map object, configured to use the OS Maps API. It will be initially centred on Abingdon town centre, with a zoom level of 18. It will use the 'Road' style, which is one of the default styles offered by this service. This style shows building divisions very clearly, which is potentially of relevance when also working with address data.

In [3]:
# Function to create ipyleaflet tile layers
def createOSMapsTileLayer(layertype):

    params = urlencode({'key': key,
                        'service': 'WMTS',
                        'request': 'GetTile',
                        'version': '2.0.0',
                        'height': 256,
                        'width': 256,
                        'outputFormat': 'image/png',
                        'style': 'default',
                        'layer': layertype,
                        'tileMatrixSet': 'EPSG:3857',
                        'tileMatrix': '{z}',
                        'tileRow': '{y}',
                        'tileCol': '{x}'})

    # OS Data Hub base path - https://api.os.uk
    # OS Maps API WMTS end point path - /maps/raster/v1/wmts?
    url = unquote(f'https://api.os.uk/maps/raster/v1/wmts?{params}')

    tileLayer = {'url': url,
               'min_zoom': 7,
               'max_zoom': 20,
               'attribution': f'Contains OS data &copy; Crown copyright and database rights {datetime.now().year}'}

    return tileLayer

In [4]:
# Create custom ipyleaflet TileLayer for the OS Maps API WMTS resource
os_maps_api_road = createOSMapsTileLayer('Road_3857')


In [5]:
# Create ipyleaflet Map
centreOfMap = [51.671251 , -1.2829059]  # Abingdon-upon-Thames
m = Map(basemap=os_maps_api_road,
        center=centreOfMap,
        zoom=18)  # Zoom level 18

In [6]:
# Add an image overlay, placing the OS logo in the bottom left corner.
oslogo_url = 'https://raw.githubusercontent.com/OrdnanceSurvey/os-api-branding/master/img/os-logo-maps.png'
oslogo_img = IPython.display.Image(oslogo_url, width = 300)
oslogoWidget = Image(value=oslogo_img.data, format='png', width=100, height=30)
widgetControl = WidgetControl(widget=oslogoWidget, position='bottomleft')
m.add_control(widgetControl)


This next function takes a list of feature data and converts them to icons that will then be added to an ipyleaflet layer. The icons, themselves, use the Font-Awesome library, which can be referenced at https://fontawesome.com/v4.7.0/icons

The 'locOffset' parameter has been introduced, here, so that we can still meaningfully visualise DPA and LPI point when they're otherwise co-incident.

In [7]:
def createPlacesAPIAweIconLayer(simplifiedlist, iname, mcolour, icolour, locOffset):
    
    markers = []

    try:
        for d in simplifiedlist:
        
            thisIcon = AwesomeIcon(name=iname, marker_color=mcolour, icon_color=icolour,spin=False)
            thisMarker = Marker(icon=thisIcon, location=(d["LAT"]-locOffset,d["LNG"]-locOffset), opacity=1, visible=True)
            markers.append(thisMarker)
    except:
        raise Exception("Error encountered whilst creating marker objects.  Check that the received JSON data is correct!")

    # Finally, add the results to the map as a complete marker cluster
    m.add_layer(MarkerCluster(markers=tuple(markers)))

Build our request URL and request JSON data from the OS Places API.

The OS Places API offers a lot of options in regards to how the address data is queried.  Area based queries can use a bounding box, polygon extent or radius, with the latter being used by this example.  Alternatively, free text, postcode, UPRN and 'nearest' queries can also be undertaken.  See https://osdatahub.os.uk/docs/places/technicalSpecification for details.

In [8]:


base_osplaces_url = 'https://api.os.uk/search/places/v1/'
osplaces_path = 'radius?point=449687,197185&radius=100'
full_osplaces_url = base_osplaces_url + osplaces_path + '&format=json&srs=BNG&dataset=DPA,LPI&output_srs=WGS84&key=' + key

try:
    response = requests.get(full_osplaces_url)
except:
    raise Exception("Unable to query and process Places API.")

try:
    data = response.json()
except:
    raise Exception("Unable to convert the returned payload to JSON.")




Now simplify the returned address data into a simple feature list, then convert that into icon layers that can be displayed on the map.

In [9]:
resultslist = data['results'].copy()  # get just the list of results
simplifiedlistDPA, simplifiedlistLPI = [],[] ## simplified lists for DPA and LPI records

for i in range(len(resultslist)):    # For the Places API, each result is a one-entry dictionary, with a further dictionary nested within.
    outerDict = resultslist[i]

    if "DPA" in outerDict:      # found a DPA record
        simplifiedlistDPA.append(outerDict["DPA"])
    elif "LPI" in outerDict:    # found an LPI record
        simplifiedlistLPI.append(outerDict["LPI"])
    else:
        raise Exception("Error:  Unknown dataset found.  Only DPA and LPI are supported.")

# Now create DPA and/or LPI map layers
if simplifiedlistDPA:
    createPlacesAPIAweIconLayer(simplifiedlistDPA, 'map-marker', 'green', 'white', 0.00001)

if simplifiedlistLPI:
    createPlacesAPIAweIconLayer(simplifiedlistLPI, 'institution', 'red', 'black', 0)

In [10]:

m

Map(center=[51.671251, -1.2829059], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…